In [1]:
import requests
import json
from datetime import datetime
import time
import os

# from dotenv import load_dotenv
# load_dotenv()

# For quick testing (without .env file)
ALICE_API_KEY = "alice_key_123"
BOB_API_KEY = "bob_key_456"
CHARLIE_API_KEY = "charlie_key_789"
base_url = "http://localhost:8000"

print("🔑 API Keys:")
print(f"   Alice: {ALICE_API_KEY}")
print(f"   Bob: {BOB_API_KEY}")
print(f"   Charlie: {CHARLIE_API_KEY}")
print(f"   Base URL: {base_url}")

🔑 API Keys:
   Alice: alice_key_123
   Bob: bob_key_456
   Charlie: charlie_key_789
   Base URL: http://localhost:8000


In [2]:
# headers = {"X-API-Key": BOB_API_KEY}
headers = {"X-API-Key": ALICE_API_KEY}

In [3]:
def print_response(response):
    """Pretty print API responses"""
    print(f"Status Code: {response.status_code}")
    print(f"Headers: {dict(response.headers)}")
    print("Response Body:")
    try:
        print(json.dumps(response.json(), indent=2))
    except:
        print(response.text)
    print("\n" + "="*50 + "\n")

In [4]:
# Test if the API is running
try:
    response = requests.get(f"{base_url}/docs", headers=headers)
    if response.status_code == 200:
        print("✅ API is running! You can view the docs at:")
        print(f"   {base_url}/docs")
    else:
        print(f"❌ API returned status code: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to API. Make sure the backend server is running.")
    print("   Run: uvicorn main:app --reload --host 0.0.0.0 --port 8000")

✅ API is running! You can view the docs at:
   http://localhost:8000/docs


In [7]:
# Test GET /users/me
print("Testing GET /users/me...")
response = requests.get(f"{base_url}/users/me", headers={"X-API-Key": CHARLIE_API_KEY})
print_response(response)

# Store user info for later tests
if response.status_code == 200:
    user_data = response.json()
    print(f"✅ User found: {user_data['name']} (ID: {user_data['id']})")
else:
    print("❌ Failed to get user information")

Testing GET /users/me...
Status Code: 200
Headers: {'date': 'Sun, 10 Aug 2025 16:39:58 GMT', 'server': 'uvicorn', 'content-length': '25', 'content-type': 'application/json'}
Response Body:
{
  "id": 3,
  "name": "Charlie"
}


✅ User found: Charlie (ID: 3)


In [10]:
# Test GET /threads/me
print("Testing GET /threads/me...")
response = requests.get(f"{base_url}/threads/me", headers={"X-API-Key": BOB_API_KEY})
print_response(response)

# Store thread info for later tests
if response.status_code == 200:
    thread_data = response.json()
    print(f"✅ Thread found: ID {thread_data['id']}")
    print(f"   Number of messages: {len(thread_data['messages'])}")
    
    # Display existing messages
    if thread_data['messages']:
        print("\n📝 Existing messages:")
        for i, msg in enumerate(thread_data['messages'], 1):
            sender = "👤 User" if msg['is_from_user'] else "🤖 Bot"
            print(f"   {i}. {sender}: {msg['content']}")
else:
    print("❌ Failed to get thread information")

Testing GET /threads/me...
Status Code: 200
Headers: {'date': 'Sun, 10 Aug 2025 16:40:57 GMT', 'server': 'uvicorn', 'content-length': '541', 'content-type': 'application/json'}
Response Body:
{
  "id": 2,
  "messages": [
    {
      "id": 3,
      "content": "Hello Bob! How can I help you today?",
      "is_from_user": false,
      "created_at": "2025-08-10T16:38:47.640787"
    },
    {
      "id": 4,
      "content": "I'm here to assist you with any questions.",
      "is_from_user": false,
      "created_at": "2025-08-10T16:38:47.640790"
    },
    {
      "id": 7,
      "content": "Hello! This is a test message from the notebook.",
      "is_from_user": true,
      "created_at": "2025-08-10T16:40:48.348537"
    },
    {
      "id": 8,
      "content": "Thank you for the information. How can I be of service?",
      "is_from_user": false,
      "created_at": "2025-08-10T16:40:48.353674"
    }
  ]
}


✅ Thread found: ID 2
   Number of messages: 4

📝 Existing messages:
   1. 🤖 Bot: Hel

In [9]:
# Test POST /messages
print("Testing POST /messages...")

# Test message data
test_message = {
    "content": "Hello! This is a test message from the notebook."
}

print(f"Sending message: {test_message['content']}")
response = requests.post(
    f"{base_url}/messages",
    json=test_message,
    headers={"Content-Type": "application/json", "X-API-Key": BOB_API_KEY}
)
print_response(response)

if response.status_code == 200:
    message_data = response.json()
    print("✅ Message sent successfully!")
    print(f"   User message ID: {message_data['user_message']['id']}")
    print(f"   Bot response ID: {message_data['bot_message']['id']}")
    print(f"   Bot response: {message_data['bot_message']['content']}")
else:
    print("❌ Failed to send message")

Testing POST /messages...
Sending message: Hello! This is a test message from the notebook.
Status Code: 200
Headers: {'date': 'Sun, 10 Aug 2025 16:40:47 GMT', 'server': 'uvicorn', 'content-length': '302', 'content-type': 'application/json'}
Response Body:
{
  "user_message": {
    "id": 7,
    "content": "Hello! This is a test message from the notebook.",
    "is_from_user": true,
    "created_at": "2025-08-10T16:40:48.348537"
  },
  "bot_message": {
    "id": 8,
    "content": "Thank you for the information. How can I be of service?",
    "is_from_user": false,
    "created_at": "2025-08-10T16:40:48.353674"
  }
}


✅ Message sent successfully!
   User message ID: 7
   Bot response ID: 8
   Bot response: Thank you for the information. How can I be of service?


In [11]:
# Test multiple messages
test_messages = [
    "How are you today?",
    "What's the weather like?",
    "Tell me a joke",
    "What can you help me with?"
]

print("Testing multiple messages...")
for i, message_content in enumerate(test_messages, 1):
    print(f"\n--- Message {i}: {message_content} ---")
    
    response = requests.post(
        f"{base_url}/messages",
        json={"content": message_content},
        headers={"Content-Type": "application/json", "X-API-Key": CHARLIE_API_KEY}
    )
    
    if response.status_code == 200:
        data = response.json()
        print(f"✅ User: {data['user_message']['content']}")
        print(f"🤖 Bot: {data['bot_message']['content']}")
    else:
        print(f"❌ Failed to send message {i}")
    
    # Small delay between messages
    time.sleep(0.5)

Testing multiple messages...

--- Message 1: How are you today? ---
✅ User: How are you today?
🤖 Bot: Thank you for the information. How can I be of service?

--- Message 2: What's the weather like? ---
✅ User: What's the weather like?
🤖 Bot: I understand what you're saying. How can I help you further?

--- Message 3: Tell me a joke ---
✅ User: Tell me a joke
🤖 Bot: I understand what you're saying. How can I help you further?

--- Message 4: What can you help me with? ---
✅ User: What can you help me with?
🤖 Bot: Thanks for sharing that with me. Is there anything specific you'd like to know?


In [12]:
# Verify message persistence
print("Verifying message persistence...")
response = requests.get(f"{base_url}/threads/me", headers={"X-API-Key": CHARLIE_API_KEY})

if response.status_code == 200:
    thread_data = response.json()
    print(f"✅ Thread has {len(thread_data['messages'])} total messages")
    
    print("\n📝 Complete conversation history:")
    for i, msg in enumerate(thread_data['messages'], 1):
        sender = "👤 User" if msg['is_from_user'] else "🤖 Bot"
        timestamp = msg['created_at'][:19]  # Show just date and time
        print(f"   {i:2d}. [{timestamp}] {sender}: {msg['content']}")
else:
    print("❌ Failed to verify message persistence")

Verifying message persistence...
✅ Thread has 10 total messages

📝 Complete conversation history:
    1. [2025-08-10T16:38:47] 🤖 Bot: Hello Charlie! How can I help you today?
    2. [2025-08-10T16:38:47] 🤖 Bot: I'm here to assist you with any questions.
    3. [2025-08-10T16:41:24] 👤 User: How are you today?
    4. [2025-08-10T16:41:24] 🤖 Bot: Thank you for the information. How can I be of service?
    5. [2025-08-10T16:41:24] 👤 User: What's the weather like?
    6. [2025-08-10T16:41:24] 🤖 Bot: I understand what you're saying. How can I help you further?
    7. [2025-08-10T16:41:25] 👤 User: Tell me a joke
    8. [2025-08-10T16:41:25] 🤖 Bot: I understand what you're saying. How can I help you further?
    9. [2025-08-10T16:41:25] 👤 User: What can you help me with?
   10. [2025-08-10T16:41:25] 🤖 Bot: Thanks for sharing that with me. Is there anything specific you'd like to know?


In [10]:
# Test error cases
print("Testing error cases...")

# Test 1: Empty message
print("\n--- Test 1: Empty message ---")
response = requests.post(
    f"{base_url}/messages",
    json={"content": ""},
    headers={"Content-Type": "application/json"}
)
print(f"Status: {response.status_code}")
print(f"Response: {response.text}")

# Test 2: Missing content field
print("\n--- Test 2: Missing content field ---")
response = requests.post(
    f"{base_url}/messages",
    json={},
    headers={"Content-Type": "application/json"}
)
print(f"Status: {response.status_code}")
print(f"Response: {response.text}")

# Test 3: Non-existent endpoint
print("\n--- Test 3: Non-existent endpoint ---")
response = requests.get(f"{base_url}/nonexistent")
print(f"Status: {response.status_code}")
print(f"Response: {response.text}")

Testing error cases...

--- Test 1: Empty message ---
Status: 200
Response: {"user_message":{"id":37,"content":"","is_from_user":true,"created_at":"2025-08-10T15:43:34.309831"},"bot_message":{"id":38,"content":"That sounds fascinating! I'd love to hear more details.","is_from_user":false,"created_at":"2025-08-10T15:43:34.311534"}}

--- Test 2: Missing content field ---
Status: 422
Response: {"detail":[{"type":"missing","loc":["body","content"],"msg":"Field required","input":{},"url":"https://errors.pydantic.dev/2.5/v/missing"}]}

--- Test 3: Non-existent endpoint ---
Status: 404
Response: {"detail":"Not Found"}
